In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 45.5 MB/s eta 0:00:00


In [3]:
import pennylane as qml
from pennylane import numpy as np

In [11]:
x_original = [1.0, 2.0, 3.0]
y_original = [2.0, 4.0, 6.0]

# Standardizing the input values to lie in the range -pi to +pi

In [12]:
x_scale = [xi*(np.pi/2)/max(x_original) for xi in x_original]
y_scale = [yi*(np.pi/2)/max(y_original) for yi in y_original]

In [13]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit(x, theta):
  qml.RY(x, wires=0)
  qml.RY(theta[0], wires=0) # represents w

  qml.RZ(theta[1], wires=0) # represents b


  return qml.expval(qml.PauliZ(0))


In [10]:
theta = np.array([0.0, 0.0], requires_grad=True)
opt = qml.GradientDescentOptimizer(stepsize=0.1)

for epoch in range(100):
  y_pred = [circuit(xi, theta) for xi in x_scale]

  loss = np.mean((np.array(y_pred) - np.array(y_scale))**2)

  theta = opt.step(lambda t: loss, theta)

  if epoch %10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss}, Theta: {theta}")




Epoch: 0, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 10, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 20, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 30, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 40, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 50, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 60, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 70, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 80, Loss: 0.9613607519996069, Theta: [0. 0.]
Epoch: 90, Loss: 0.9613607519996069, Theta: [0. 0.]


# Classification of Cats/Dogs using quantum machine learning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
import pennylane as qml

# ResNet setup
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove FC layer
for param in resnet.parameters():
    param.requires_grad = False

# Quantum setup
n_qubits = 4
n_layers = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    # Batch-aware angle encoding (no decorator needed)
    for i in range(n_qubits):
        qml.RX(inputs[:, i], wires=i)  # Critical: [batch, feature] indexing

    # Variational layers
    for layer in range(n_layers):
        for i in range(n_qubits):
            qml.Rot(*weights[layer, i], wires=i)
        for i in range(n_qubits):
            qml.CNOT(wires=[i, (i+1) % n_qubits])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (n_layers, n_qubits, 3)}
qlayer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)

# Hybrid model
class QuantumResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet
        self.reduce = nn.Linear(512, n_qubits)  # Trainable reduction
        self.quantum = qlayer
        self.fc = nn.Linear(n_qubits, num_classes)

    def forward(self, x):
        features = self.resnet(x)
        features = torch.flatten(features, 1)  # Flatten to [batch, 512]
        reduced = self.reduce(features)  # Shape: [batch, n_qubits]
        quantum_output = self.quantum(reduced)  # Process entire batch
        return self.fc(quantum_output)

# Training setup remains unchanged
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QuantumResNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Accuracy: {100 * correct / total:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 94.3MB/s]


100%|██████████| 170M/170M [00:02<00:00, 74.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/30, Accuracy: 46.85%
Epoch 2/30, Accuracy: 52.85%
Epoch 3/30, Accuracy: 60.23%


In [ ]:
torch.save(model.state_dict(), "quantum_resnet.pth")

In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),           # Resize to 224x224 (ResNet requirement)
    transforms.ToTensor(),            # Convert to tensor
    transforms.Normalize(             # Use same normalization as training
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
model = QuantumResNet(num_classes=10).to(device)
model.load_state_dict(torch.load("quantum_resnet.pth"))  # Load saved weights
model.eval()  # Set to evaluation mode

In [ ]:
from PIL import Image

# Load and preprocess an image
image = Image.open("your_image.jpg")  # Replace with your image path
input_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension

# Make prediction
with torch.no_grad():
    outputs = model(input_tensor)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

# Map class index to CIFAR-10 label
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']
print(f"Predicted class: {cifar10_classes[predicted_class]}")

In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"Test Accuracy: {accuracy * 100:.2f}%")